In [1]:
import ccxt
import os
from dotenv import load_dotenv
import time
import pandas as pd
import numpy as np
from datetime import date, datetime, timezone, tzinfo

In [2]:
load_dotenv()

# Retrieve API keys from environment variables
xP_KEY = os.getenv('xP_KEY')
xP_SECRAT = os.getenv('xP_SECRET')

kC_KEY = os.getenv('kC_KEY')
kC_SECRET = os.getenv('kC_SECRAT')
kC_PASSWORD = os.getenv('kC_PASSWORD')


In [ ]:
# Initialize the KuCoin exchange object
kucoin = ccxt.kucoin({
    'enableRateLimit': True,
    'apiKey': kC_KEY,
    'secret': kC_SECRET,
    'password': kC_PASSWORD
})

phemex = ccxt.phemex({
    'enableRateLimit': True,
    'apiKey': xP_KEY,
    'secret': xP_SECRAT
})


# Dialy SMA 20 Days

In [20]:
symbol='uBTCUSD'
pos_size = 1
params = {'timeInForce','PostOnly'} 

#! FIND DAILY SMA PER 20 DAYS

def daily_sma():
    print("starting indis ...")

    timeframe = '1d'
    num_bars = 100
    bars = phemex.fetch_ohlcv(symbol=symbol, timeframe=timeframe, limit=num_bars)
    # print(" Got the Bars: ",bars)
    data_20days = pd.DataFrame(bars, columns=['timestep', 'open', 'high', 'low', 'close', 'volume'])
    data_20days['date'] = pd.to_datetime(data_20days['timestep'], unit='ms')
    data_20days.drop(columns=['timestep'], inplace=True)
    data_20days['SMA_20'] = data_20days['close'].rolling(window=20).mean()
    return data_20days

daily_sma()

starting indis ...


,open,high,low,close,volume,date,SMA_20
20,71542.2,73769.2,71396.9,73134.8,348812.0,2024-03-13,63271.355
21,73125.4,73899.9,68599.9,71433.9,404598.0,2024-03-14,64304.810
22,71441.4,72482.9,65562.2,69544.7,407039.0,2024-03-15,65202.795
23,69551.4,70104.5,64800.0,65314.6,300623.0,2024-03-16,65880.755
24,65378.0,68917.7,64578.4,68439.7,296720.0,2024-03-17,66575.130
...,...,...,...,...,...,...,...
95,66249.5,66249.5,66249.5,66249.5,0.0,2024-05-29,64637.505
96,66249.5,66249.5,66249.5,66249.5,0.0,2024-05-30,64892.940
97,66249.5,66249.5,66249.5,66249.5,0.0,2024-05-31,65053.235
98,66249.5,66249.5,66249.5,66249.5,0.0,2024-06-01,65327.480


#  15 Minutes SMA

In [70]:
def sma_15min():
    print("starting indis ...")

    timeframe = '15m'
    num_bars = 100
    bars = phemex.fetch_ohlcv(symbol=symbol, timeframe=timeframe, limit=num_bars)
    # # print(" Got the Bars: ",bars)
    data_15m = pd.DataFrame(bars, columns=['timestep', 'open', 'high', 'low', 'close', 'volume'])
    data_15m['date'] = pd.to_datetime(data_15m['timestep'], unit='ms')
    data_15m.drop(columns=['timestep'], inplace=True)
    data_15m['SMA_20'] = data_15m['close'].rolling(window=20).mean()
    return data_15m

sma_15min()

starting indis ...


,open,high,low,close,volume,date,SMA_20
0,66249.5,66249.5,66249.5,66249.5,0.0,2024-06-02 10:15:00,NaN
1,66249.5,66249.5,66249.5,66249.5,0.0,2024-06-02 10:30:00,NaN
2,66249.5,66249.5,66249.5,66249.5,0.0,2024-06-02 10:45:00,NaN
3,66249.5,66249.5,66249.5,66249.5,0.0,2024-06-02 11:00:00,NaN
4,66249.5,66249.5,66249.5,66249.5,0.0,2024-06-02 11:15:00,NaN
...,...,...,...,...,...,...,...
95,66249.5,66249.5,66249.5,66249.5,0.0,2024-06-03 10:00:00,66249.5
96,66249.5,66249.5,66249.5,66249.5,0.0,2024-06-03 10:15:00,66249.5
97,66249.5,66249.5,66249.5,66249.5,0.0,2024-06-03 10:30:00,66249.5
98,66249.5,66249.5,66249.5,66249.5,18641.0,2024-06-03 10:45:00,66249.5
